In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def convert_to_datetime(date_string):
    norsk_to_engelsk = {
        'mandag': 'Monday', 'tirsdag': 'Tuesday', 'onsdag': 'Wednesday',
        'torsdag': 'Thursday', 'fredag': 'Friday', 'lørdag': 'Saturday',
        'søndag': 'Sunday', 'januar': 'January', 'februar': 'February',
        'mars': 'March', 'april': 'April', 'mai': 'May', 'juni': 'June',
        'juli': 'July', 'august': 'August', 'september': 'September',
        'oktober': 'October', 'november': 'November', 'desember': 'December'
    }
    for norsk, engelsk in norsk_to_engelsk.items():
        date_string = date_string.replace(norsk, engelsk)
    return datetime.strptime(date_string + ' 2024', '%A %d. %B %Y')

def scrape_numbers():
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    hovedtall = [int(li.text.strip()) for li in soup.select('ul li.h-10.w-10.rounded-full')][:5]
    stjernetall = [int(li.text.strip()) for li in soup.select('ul li.h-10.w-10.rounded-full')][5:]
    
    date = soup.select_one('span.font-medium.font-jubel').text.strip()
    
    return date, hovedtall, stjernetall

# Sett opp Chrome-opsjoner for headless-modus
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920x1080")

# Sett opp Selenium WebDriver med Chrome-opsjoner
driver = webdriver.Chrome(options=chrome_options)
driver.get("https://www.norsk-tipping.no/lotteri/eurojackpot/resultater")

results = []

while True:
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'ul li.h-10.w-10.rounded-full'))
        )
        
        date, hovedtall, stjernetall = scrape_numbers()
        results.append({'Dato': date, 'Hovedtall': hovedtall, 'Stjernetall': stjernetall})
        print(f"Scraped: {date}")
        
        previous_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "results-selected-draw-previous"))
        )
        previous_button.click()
        
    except TimeoutException:
        print("Kan ikke klikke mer eller siden lastet ikke. Avslutter.")
        break

driver.quit()

# Opprett DataFrame
results_df = pd.DataFrame(results)

# Konverter 'Dato' til datetime
results_df['Dato'] = results_df['Dato'].apply(convert_to_datetime)

# Sorter DataFrame etter dato i synkende rekkefølge
sorted_df = results_df.sort_values(by='Dato', ascending=True)

# Vis sortert DataFrame
print(sorted_df)

Scraped: fredag 15. november
Scraped: tirsdag 12. november
Scraped: fredag 8. november
Scraped: tirsdag 5. november
Scraped: fredag 1. november
Scraped: tirsdag 29. oktober
Scraped: fredag 25. oktober
Scraped: tirsdag 22. oktober
Scraped: fredag 18. oktober
Scraped: tirsdag 15. oktober
Scraped: fredag 11. oktober
Scraped: tirsdag 8. oktober
Scraped: fredag 4. oktober
Scraped: tirsdag 1. oktober
Scraped: fredag 27. september
Scraped: tirsdag 24. september
Scraped: fredag 20. september
Scraped: tirsdag 17. september
Scraped: fredag 13. september
Scraped: tirsdag 10. september
Scraped: fredag 6. september
Scraped: tirsdag 3. september
Scraped: fredag 30. august
Scraped: tirsdag 27. august
Scraped: fredag 23. august
Scraped: tirsdag 20. august
Scraped: fredag 16. august
Scraped: tirsdag 13. august
Scraped: fredag 9. august
Scraped: tirsdag 6. august
Scraped: fredag 2. august
Kan ikke klikke mer eller siden lastet ikke. Avslutter.
         Dato             Hovedtall Stjernetall
30 2024-08-0

In [3]:
# Assuming sorted_df is your DataFrame

# Define the set of draws
draws_set = list(range(len(sorted_df)))
draws_output = f"# Define the set of draws\nset DRAWS := {' '.join(map(str, draws_set))};"

# Generate Hovedtall output
hovedtall_output = "# Hovedtall data (5 numbers per draw)\nparam Hovedtall : 1 2 3 4 5 :="
for index, row in sorted_df.iterrows():
    hovedtall_numbers = ' '.join(f"{num:4d}" for num in row['Hovedtall'])
    hovedtall_output += f"\n   {index:2d} {hovedtall_numbers}"

# Generate Stjernetall output
stjernetall_output = "# Stjernetall data (2 numbers per draw)\nparam Stjernetall : 1 2 :="
for index, row in sorted_df.iterrows():
    stjernetall_numbers = ' '.join(f"{num:4d}" for num in row['Stjernetall'])
    stjernetall_output += f"\n   {index:2d} {stjernetall_numbers}"

# Combine all outputs
final_output = f"{draws_output}\n\n{hovedtall_output}\n;\n\n{stjernetall_output}\n;"

# Print or save the final output
print(final_output)

# Optionally, save to a file
# with open('lottery_data.dat', 'w') as f:
#     f.write(final_output)


# Define the set of draws
set DRAWS := 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30;

# Hovedtall data (5 numbers per draw)
param Hovedtall : 1 2 3 4 5 :=
   30   14   20   26   30   31
   29    4    9   22   32   35
   28   15   18   25   29   35
   27    6    9   33   34   50
   26    4   11   16   46   50
   25   25   28   29   31   33
   24   13   21   22   26   48
   23   15   24   29   33   39
   22    8   11   25   31   48
   21    5   17   23   36   37
   20    7   11   27   42   45
   19    7   10   31   41   46
   18    2    3   17   40   44
   17    1    3   13   24   44
   16    9   17   19   26   39
   15    3   13   34   41   43
   14    6   15   25   29   41
   13   17   37   42   45   50
   12    4   16   27   34   44
   11   13   29   42   44   48
   10    8   11   23   44   45
    9    9   20   38   44   45
    8    2    4   23   30   40
    7    4   32   36   38   47
    6    2   19   36   42   50
    5    3   17   26   30   49
 

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def convert_to_datetime(date_string):
    norsk_to_engelsk = {
        'mandag': 'Monday', 'tirsdag': 'Tuesday', 'onsdag': 'Wednesday',
        'torsdag': 'Thursday', 'fredag': 'Friday', 'lørdag': 'Saturday',
        'søndag': 'Sunday', 'januar': 'January', 'februar': 'February',
        'mars': 'March', 'april': 'April', 'mai': 'May', 'juni': 'June',
        'juli': 'July', 'august': 'August', 'september': 'September',
        'oktober': 'October', 'november': 'November', 'desember': 'December'
    }
    for norsk, engelsk in norsk_to_engelsk.items():
        date_string = date_string.replace(norsk, engelsk)
    return datetime.strptime(date_string + ' 2024', '%A %d. %B %Y')

def scrape_numbers():
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    all_numbers = [int(li.text.strip()) for li in soup.select('ul li.h-10.w-10.rounded-full')]
    hovedtall = all_numbers[:7]
    stjernetall = all_numbers[7:]
    
    date = soup.select_one('span.font-medium.font-jubel').text.strip()
    
    return date, hovedtall, stjernetall

# Set up Chrome options for headless mode
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920x1080")

# Set up Selenium WebDriver with Chrome options
driver = webdriver.Chrome(options=chrome_options)
driver.get("https://www.norsk-tipping.no/lotteri/lotto/resultater")

results = []

while True:
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'ul li.h-10.w-10.rounded-full'))
        )
        
        date, hovedtall, stjernetall = scrape_numbers()
        results.append({'Dato': date, 'Hovedtall': hovedtall, 'Stjernetall': stjernetall})
        print(f"Scraped: {date}")
        
        previous_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "results-selected-draw-previous"))
        )
        previous_button.click()
        
    except TimeoutException:
        print("Can't click anymore or page didn't load. Exiting.")
        break

driver.quit()

# Create DataFrame
results_df = pd.DataFrame(results)

# Convert 'Dato' to datetime
results_df['Dato'] = results_df['Dato'].apply(convert_to_datetime)

# Sort DataFrame by date in ascending order
sorted_df = results_df.sort_values(by='Dato', ascending=True)

# Display sorted DataFrame
print(sorted_df)


Scraped: lørdag 9. november
Scraped: lørdag 2. november
Scraped: lørdag 26. oktober
Scraped: lørdag 19. oktober
Scraped: lørdag 12. oktober
Scraped: lørdag 5. oktober
Scraped: lørdag 28. september
Scraped: lørdag 21. september
Scraped: lørdag 14. september
Scraped: lørdag 7. september
Scraped: lørdag 31. august
Scraped: lørdag 24. august
Scraped: lørdag 17. august
Scraped: lørdag 10. august
Scraped: lørdag 3. august
Can't click anymore or page didn't load. Exiting.
         Dato                    Hovedtall Stjernetall
14 2024-08-03  [9, 16, 18, 24, 26, 31, 33]        [29]
13 2024-08-10    [6, 7, 9, 13, 19, 20, 33]        [16]
12 2024-08-17    [4, 7, 8, 20, 32, 33, 34]         [3]
11 2024-08-24    [1, 4, 5, 18, 21, 24, 33]        [17]
10 2024-08-31   [3, 5, 12, 15, 16, 21, 29]        [18]
9  2024-09-07    [1, 4, 8, 11, 12, 19, 28]        [10]
8  2024-09-14  [3, 10, 15, 17, 21, 23, 32]        [31]
7  2024-09-21    [1, 2, 6, 20, 25, 28, 33]        [18]
6  2024-09-28   [3, 8, 15, 24, 29, 

In [10]:
# Assuming sorted_df is your DataFrame

# Define the set of draws
draws_set = list(range(len(sorted_df)))
draws_output = f"# Define the set of draws\nset DRAWS := {' '.join(map(str, draws_set))};"

# Generate Hovedtall output
hovedtall_output = "# Hovedtall data (7 numbers per draw)\nparam Hovedtall : 1 2 3 4 5 6 7 :="
for index, row in sorted_df.iterrows():
    hovedtall_numbers = ' '.join(f"{num:4d}" for num in row['Hovedtall'])
    hovedtall_output += f"\n   {index:2d} {hovedtall_numbers}"

# Generate Stjernetall output
stjernetall_output = "# Stjernetall data (1 number per draw)\nparam Stjernetall :="
for index, row in sorted_df.iterrows():
    stjernetall_number = row['Stjernetall'][0]  # Assuming Stjernetall is still a list
    stjernetall_output += f"\n   {index:2d} {stjernetall_number:4d}"

# Combine all outputs
final_output = f"{draws_output}\n\n{hovedtall_output}\n;\n\n{stjernetall_output}\n;"

# Print or save the final output
print(final_output)

# Optionally, save to a file
# with open('lottery_data.dat', 'w') as f:
#     f.write(final_output)


# Define the set of draws
set DRAWS := 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14;

# Hovedtall data (7 numbers per draw)
param Hovedtall : 1 2 3 4 5 6 7 :=
   14    9   16   18   24   26   31   33
   13    6    7    9   13   19   20   33
   12    4    7    8   20   32   33   34
   11    1    4    5   18   21   24   33
   10    3    5   12   15   16   21   29
    9    1    4    8   11   12   19   28
    8    3   10   15   17   21   23   32
    7    1    2    6   20   25   28   33
    6    3    8   15   24   29   33   34
    5    3    7   10   17   19   21   29
    4    6   11   13   16   17   21   34
    3    2    3   10   13   14   27   29
    2    7    8   13   20   21   30   31
    1    2    7   13   23   27   30   33
    0    4    6   13   19   21   25   33
;

# Stjernetall data (1 number per draw)
param Stjernetall :=
   14   29
   13   16
   12    3
   11   17
   10   18
    9   10
    8   31
    7   18
    6    5
    5   25
    4    3
    3   22
    2    1
    1   22
    0    5
;
